# CRISP-DM: Data Understanding

## Imports

In [1]:
import platform
import os
import gc
import random
import numpy as np
import pandas as pd
from datetime import timedelta

import mne
mne.set_log_level('error')
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

## Collect Initial Data

**Selection**

The data used for the research is from (medically-induced) comatose pediatric patients. The selection consists of 127 EEG recordings from such patients in Erasmus MC's Pediatric Intensive Care Unit (PICU). These are selected specifically for the research by the neurologist (R. van den Berg) supervising this project, based on their availability quality and relevance. The data consists of various length recordings which are labeled with annotations, including the event that a seizure occurred.

**Storage**

The data is stored on and accessible through an anDREa workspace. The Digital Research Environment (DRE) is used by various UMCs in the Netherlands to conduct research while accounting for security, data privacy, and computational needs. The sample EEG recordings have undergone a process of some sort, after which they are made available in EDF format, accessible via a Azure Virtual Machine on the workspace in a dedicated project directory. Since patient data is stored here, there is no Internet access in the VM to minimise the risk of a data breach.

In [2]:
# Define directory paths with relevant system prefix
def define_paths(project_dir, data_dir):
    SYSTEM = platform.system()
    if SYSTEM not in ('Windows', 'Linux'):
        raise Exception(f'Unsupported platform: {SYSTEM}')
    
    disk_prefix = None

    if SYSTEM == 'Windows':
        disk_prefix = 'Z:/'
    elif SYSTEM == 'Linux':
        disk_prefix = '/media/mount/'

    project_path = os.path.join(disk_prefix, project_dir)
    data_path = os.path.join(project_path, data_dir)

    return project_path, data_path

In [3]:
# Select all EDF files in data directory
def select_edf_files(data_path):
    edf_files = [file for file in os.listdir(data_path) if file.endswith('.edf')]
    print(f"{len(edf_files)} EDF files available")

    return edf_files

In [4]:
# Define paths and select files
project_dir = '08_DeepLearning_SeizureDetection/'
project_path, data_path = define_paths(project_dir, data_dir='_01_Raw_Data/EDF/')

edf_files = select_edf_files(data_path)

127 EDF files available


**Loading**

The data is loaded with MNE, which is an open-source Python package for exploring, visualizing, and analyzing human neurophysiological data, including EEG. Loading the data can be very memory intensive. The EDF file sizes range from 7 Megabytes to 9 Gigabytes. Loading the files can require a lot of working memory if the file itself is also large in size. Since most values are represented as floating-point numbers with 6 decimal places, this causes a large memory footprint.

**Problem Handling**

To limit memory constraints, a number of solutions are applied. First of all, data is not pre-loaded but only read initially. During this, datatypes are inferred, of which only EEG channels are selected. This omits technical and other physiological data channels such as movement of the extremities, eye movement, respiration and oxygen saturation. This is possible because only EEG channels are selected for inclusion in data analysis and training the model.

If a recording is still too large to load despite applying the steps above, there is a different approach. Some files may exceed the available committed memory (75 GB), even after raising the VM's RAM from 16 GB to 64 GB. If the file is this large it means that it is a long recording (e.g. 5 days of continuous monitoring). It is important to still include large files these in the data selection, as they may include many seizures and data with labeled seizures is often in short supply. The number of seizures annotated is often related to the length of the recording. 

To still include large files in the loading is separated into two processes for the large files : seizure and non-seizure handling. The data is first read without preloading to extract seizure annotations. The data is then cropped by these annotations times, resulting in a raw object for each individual seizure, which is loaded and concatenated at a later point. Based on the number of epochs (10s segment) that fit in the seizure data, the same amount of data is loaded to get non-seizure data. E.g., if a recording consists of a total of 300 seconds of ictal data, 300 seconds of seizure-free data is loaded in segments.

**Loading a sample**

In [5]:
# Define EEG channel names for selection
eeg_channels = [
    'Fp1', 'Fp2', 'F3', 'F4',
    'F7',  'F8',  'Fz',  'C3', 
    'C4',  'Cz',  'T3',  'T4', 
    'T5',  'T6',  'P3',  'P4', 
    'Pz',  'O1',  'O2'
]

In [6]:
# Loading function
def load_raw(data_path, file, seizures_only, load):
    try:
        file_path = os.path.join(data_path, file)
        raw = mne.io.read_raw_edf(file_path, infer_types=True, include=eeg_channels, preload=False)

        if seizures_only == True: # only loads files with seizures
            # Check if the recording contains seizures
            if not any('AANVAL' in ann['description'] for ann in raw.annotations):
                raise Exception('No seizures detected in the data')
            else:
                seizure_subjects.append(file)
        
        if load == True:
            raw.load_data()

            raw.set_channel_types({ch: 'eeg' for ch in raw.ch_names})
            raw.set_eeg_reference(ref_channels='average')

        return raw
    
    except Exception as e:
        print(f"{file} did not load: {e}")
        return None

In [ ]:
# Pick random file as sample and try reading/loading until succesful
while True:
    file = random.choice(edf_files)
    print(f'Loading {file}...')

    sample = load_raw(data_path, file, seizures_only=False, load=True)

    if sample is not None:
        break
        print(f'Succesfully loaded EEG data')

In [ ]:
sample

## Describe data

### Data description report

#### Per subject

When convering the RawEDF object to a Pandas DataFrame, the float values are converted from volt to microvolt

In [ ]:
sample_df = sample.to_data_frame()
sample_df.head()

All values are float64 with 6 decimal points and there are no missing values

In [ ]:
sample_df.info()

In [ ]:
sample_df.describe()

It is also possible to generate descriptive statistics directly on the RawEDF, but this excludes mean and standard deviation

In [ ]:
sample.describe()

In [ ]:
dur = sample._data.shape[1] / sample.info['sfreq']
segment_dur = 10
start_time = random.uniform(0, dur - segment_dur)
end_time = start_time + segment_dur

In [ ]:
def plot_sample(start_time, end_times, channels, lobe):
    fig, ax = plt.subplots(figsize=[15, 5])

    for ch in channels:
        data = sample.get_data(picks=ch, tmin=start_time, tmax=end_time).T
        times = np.linspace(0, 10, num=data.shape[0])
        ax.plot(times, data * 1e5, label=ch, linewidth=1)

    plt.text(0, 1.01, '1e-5', transform=ax.transAxes)
    plt.title(f'Raw {segment_dur}s sample of {lobe} electrodes')
    plt.xlabel('Time (s)')
    plt.xticks(np.arange(0, 11, 1))
    plt.ylabel('Voltage (µV)')
    plt.legend()
    plt.show()

    #fig.savefig(f'random_sample_{channels}.png')
    print(f'Start time: {start_time} s')
    return fig


Plotting a selection of channels with matplotlib

In [ ]:
sample_fig = plot_sample(start_time, end_time, channels=['P3', 'P4', 'Pz'], lobe='parietal')

Plotting the entire EEG and select a segment with MNE visualization

In [ ]:
sample.plot(start=start_time, duration=segment_dur)

Relationship between electrodes

As EEG abnormalities can be focal, regional, or hemispheric. This could very well result in an attribute relationship between electrodes.

In [ ]:
corr = sample_df.drop(columns=['time']).corr()
plt.figure(figsize=(14, 12))
corr_matrix = sns.heatmap(corr, fmt='.2f', cmap='coolwarm', cbar_kws={'label':'Correlation'}, annot=True, linewidths=.5)
plt.title('Correlation Matrix of Electrode Values Across EEG')
plt.show()

fig = corr_matrix.get_figure()
#fig.savefig('Sample Electrode Heatmap.png')

In [ ]:
psd = sample.compute_psd()
psd = psd.plot()
#psd.savefig(f'PSD_plot.png')

#### All subjects
A number of description methods are applied to all subjects together, as it can be difficult to describe each recording individually. The DataFrame below shows information extracted from each file and summarized.

In [ ]:
def get_eeg_info(file, data_path, seizure_only):
    raw = load_raw(data_path, file, seizures_only=seizure_only, load=False)

    if raw is not None:
        seizures = raw.annotations[raw.annotations.description == 'AANVAL'] # Annotations with 'SEIZURE' as description
        raw.annotations[raw.annotations.description == 'AANVAL']
        file_path = os.path.join(data_path, file)

        try:
            min_dur = int(min(seizures.duration).round())
            max_dur = int(max(seizures.duration).round())
        except Exception:
            min_dur = np.nan
            max_dur = np.nan
            
        return {
            'Subject ID' : '_'.join(os.path.splitext(file)[0].rsplit('_', 2)[-2:]),
            'Duration' : timedelta(seconds=int(raw.times[-1])),
            'Duration (s)' : int(raw.times[-1]),
            'Number of seizures' : len(seizures),
            'Total duration of all seizures (s)' : np.sum(seizures.duration).round(),
            'Average duration of seizures (s)' : np.median(seizures.duration).round(),
            'Min duration of seizures (s)' : min_dur,
            'Max duration of seizures (s)' : max_dur,
            'Number of EEG channels' : len(raw.info['ch_names']),
            'Channel names' : raw.ch_names,
            'Sampling frequency (Hz)' : raw.info['sfreq'],
            'Highpass' : raw.info['highpass'],
            'Lowpass' : raw.info['lowpass'],        
            'File size (MB)' : int(os.path.getsize(file_path) / (1024 ** 2))
        }

columns = [
'Subject ID', 'Duration', 'Duration (s)',
'Number of seizures', 'Total duration of all seizures (s)', 'Average duration of seizures (s)',
'Min duration of seizures (s)', 'Max duration of seizures (s)', 'Number of EEG channels', 'Channel names',
'Sampling frequency (Hz)', 'Highpass', 'Lowpass', 'File size (MB)'
]

data_description_df = pd.DataFrame(columns=columns)

In [ ]:
# ecg - electrocardiogram (heart)
# eeg - electroencephalogram (brain)
# emg - electromyogram (muscles)
# eog - electrooculogram (eyes)
# misc - miscellaneous
# resp - changes in chest or abdominal movements related to breathing
# stim - stimulus / triggers
# syst - monitoring system

ch_type_dict = {
    'EEG Fp1' : 'eeg', 'EEG Fp2' : 'eeg', 'EEG F3' : 'eeg',
    'EEG F4' : 'eeg', 'EEG F7' : 'eeg', 'EEG F8' : 'eeg',
    'EEG Fz' : 'eeg', 'EEG C3' : 'eeg', 'EEG C4' : 'eeg',
    'EEG Cz' : 'eeg', 'EEG T3' : 'eeg', 'EEG T4' : 'eeg',
    'EEG T5' : 'eeg', 'EEG T6' : 'eeg', 'EEG P3' : 'eeg',
    'EEG P4' : 'eeg', 'EEG Pz' : 'eeg', 'EEG O1' : 'eeg',
    'EEG O2' : 'eeg',

    'ECG' : 'ecg', 'ECG-0' : 'ecg', 'ECG-1' : 'ecg',

    'Unspec Beweging-0' : 'emg', 'Unspec Beweging-1' : 'emg',
    'EMG Kin Beweging' : 'emg', 'EMG Kin rec' : 'emg',
    'EMG Kin lin' : 'emg', 'EMG Kin beweging' : 'emg',
    'EMG Ch7' : 'emg', 'EMG Ch8' : 'emg',

    'Unspec Ademhalin' : 'resp', 'Resp Ademhaling' : 'resp',
    'Resp RESP' : 'resp', 'Resp RESP #2' : 'resp',
    
    'SaO2 Saturatie' : 'misc', 'SaO2 HR' : 'misc',
    'SaO2 SpO2' : 'misc', 'SaO2 SPO2 #2' : 'misc',
    
    'EOG EOG' : 'eog',
    'Light Stimuli' : 'stim',

    'Unspec Sig Buf' : 'syst', 'Unspec CPU' : 'syst',
    'Unspec Mem' : 'syst', 'Unspec App CPU' : 'syst',
    'Unspec App Mem' : 'syst', 'Unspec App VMem' : 'syst',
    'Unspec Network' : 'syst', 'Unspec Dev Cntr' : 'syst',
    'Unspec Rec Cntr' : 'syst', 'Unspec Rec Crc' : 'syst',

    'BloodP ART S' : 'misc', 'BloodP ART D' : 'misc',
    'BloodP ART M' : 'misc', 'BloodP ART' : 'misc',
}

In [ ]:
seizure_files = [
 'EXP_EMC_BR2_0000210#01.edf','EXP_EMC_BR3_0000495#01.edf','EXP_EMC_BR3_0000520#01.edf',
 'EXP_EMC_M15_0000099#01.edf','EXP_EMC_M15_0000106#01.edf','EXP_EMC_M15_0000120#01.edf',
 'EXP_EMC_M15_0000139#01.edf','EXP_EMC_M15_0000151#01.edf','EXP_EMC_M15_0000241#01.edf',
 'EXP_EMC_M15_0000311#01.edf','EXP_EMC_M15_0000333#01.edf','EXP_EMC_M15_0000434#01.edf',
 'EXP_EMC_M15_0000469#01.edf','EXP_EMC_M15_0000556#01.edf','EXP_EMC_M15_0000568#01.edf',
 'EXP_EMC_M15_0000574#01.edf','EXP_EMC_M15_0000615#01.edf','EXP_EMC_M16_0000070#01.edf',
 'EXP_EMC_M16_0000109#01.edf','EXP_EMC_M16_0000110#01.edf','EXP_EMC_M16_0000111#01.edf',
 'EXP_EMC_M16_0000153#01.edf','EXP_EMC_M16_0000168#01.edf','EXP_EMC_M16_0000232#01.edf',
 'EXP_EMC_M16_0000233#01.edf','EXP_EMC_M16_0000243#01.edf','EXP_EMC_M16_0000244#01.edf',
 'EXP_EMC_M16_0000351#01.edf','EXP_EMC_M16_0000541#01.edf','EXP_EMC_M16_0000542#01.edf',
 'EXP_EMC_M16_0000555#01.edf','EXP_EMC_M16_0000579#01.edf','EXP_EMC_M16_0000580#01.edf',
 'EXP_EMC_M16_0000583#01.edf','EXP_EMC_M16_0000600#01.edf','EXP_EMC_M18_0001180#01.edf',
 'EXP_EMC_M6b_0000513#01.edf'
 ]

In [ ]:
sample

In [ ]:
seizure_subjects = []

for file in tqdm(edf_files):
    file_info = get_eeg_info(file, data_path, seizure_only=False)
    data_description_df = pd.concat([data_description_df, pd.DataFrame([file_info])], ignore_index=True)

In [ ]:
data_description_df

In [ ]:
data_description_df.to_csv('Data Description DataFrame all files.csv')

In [7]:
combined_descriptions = pd.DataFrame()

for file in tqdm(edf_files):
    try:
        raw = load_raw(data_path, file, seizures_only=False, load=False)
        raw = raw.to_data_frame()
        raw = raw.describe()
        description = raw.T
        subject_id = '_'.join(os.path.splitext(file)[0].rsplit('_', 2)[-2:])
        description.insert(0, 'subject', subject_id)
        combined_descriptions = pd.concat([combined_descriptions, description])
        combined_descriptions.drop(combined_descriptions.index[0], inplace=True)
        for col in ['mean', 'std', 'min', '25%', '50%', '75%', 'max']:
            df[col] = df[col].round(2)

    except Exception as e:
        print(e)

combined_descriptions.reset_index(inplace=True)

  0%|          | 0/127 [00:00<?, ?it/s]

 17%|█▋        | 21/127 [16:04<3:09:12, 107.10s/it]

Unable to allocate 19.6 GiB for an array with shape (20, 131652096) and data type float64


 28%|██▊       | 36/127 [31:42<3:31:44, 139.61s/it]

Unable to allocate 28.4 GiB for an array with shape (20, 190688256) and data type float64


 62%|██████▏   | 79/127 [1:27:28<2:26:58, 183.72s/it]

Unable to allocate 23.0 GiB for an array with shape (20, 154168576) and data type float64


100%|██████████| 127/127 [2:32:45<00:00, 72.17s/it]  


In [8]:
combined_descriptions

,index,Subject ID,count,mean,std,min,25%,50%,75%,max
0,time,BR2_0000210#01,2743040.0,5357.498047,3093.154631,0.000000,2678.749023,5357.498047,8036.247070,10714.996094
1,Fp1,BR2_0000210#01,2743040.0,-11.713426,78.101756,-4990.685893,-25.143572,-14.406104,-3.131762,5012.518746
2,Fp2,BR2_0000210#01,2743040.0,27.059675,79.431962,-4990.327977,10.469032,24.606699,39.639155,5012.339788
3,F3,BR2_0000210#01,2743040.0,-88.300863,74.916126,-4991.043809,-93.505455,-89.031510,-84.915480,5010.371252
4,F4,BR2_0000210#01,2743040.0,-58.152608,75.804326,-4990.149020,-67.914488,-58.787640,-49.123919,5011.087083
...,...,...,...,...,...,...,...,...,...,...
2474,T6,M6b_0000513#01,78094250.0,0.033917,30.758432,-2286.793317,-7.858396,-0.228885,7.553216,5000.000000
2475,P3,M6b_0000513#01,78094250.0,0.107160,339.379215,-5000.000000,-206.225681,0.076295,207.751583,5000.000000
2476,P4,M6b_0000513#01,78094250.0,0.084863,374.538066,-5000.000000,-231.403067,-0.381476,232.013428,5000.000000
2477,O1,M6b_0000513#01,78094250.0,0.039952,21.071800,-2314.869917,-7.553216,0.076295,7.553216,2598.992905


**Relationships Between Attributes**

Duration and Seizures:
As mentioned in the loading process, the number of seizures in an EEG is often related to the duration. To verify if this the case for the data available, a few values are calculated. When calculating the Pearson Correlation Coefficient between duration and number of seizures, this gives a number of 0.76, indicating there is a strong relationship between the two. This is calculated on data where one or more seizures occur. When calculating it on the entire dataset, including recording with no seizures, the PCC is 0.73.

The scatter plot shows the spread of EEG duration against number of seizures. This visualizes an important relationship between the two variables and indeed shows that the longest recordings contain the most seizures, although, only a few values account for a high number of seizures. When detecting outliers with the Z-score method, only the recording with 267 seizures is seen as an outlier.

In [ ]:
cols = ['Duration (s)', 'Seizures', 'Sum (s)', 'Avg (s)', 'Min (s)', 'Max (s)']
data_description_df[cols] = data_description_df[cols].apply(pd.to_numeric)
data_description_df['Duration (h)'] = data_description_df['Duration (s)'] / 3600

In [ ]:
sns.lmplot(x='Duration (h)' , y='Number of seizures', data=data_description_df, height=6, aspect=1)
plt.title('Relationship Between EEG Duration and Number of Seizures')
plt.xlabel('EEG duration in hours')
plt.ylabel('Number of seizures')
plt.show()

In [ ]:
corr = np.corrcoef(data_description_df['Duration (s)'], data_description_df['Number of seizures'])[0,1]
print(f'Correlation Coefficient: {corr}')

In [ ]:
z_scores = np.abs((data_description_df['Number of seizures'] - data_description_df['Number of seizures'].mean()) / data_description_df['Number of seizures'].std())
threshold = 3
outliers = data_description_df[z_scores > threshold]
outliers

### Data Quality

In [ ]:
def consistency_check(file, data_path, seizure_only):
    raw = load_raw(data_path, file, seizures_only=seizure_only, load=False)

    if raw is not None:
        seizures = raw.annotations[raw.annotations.description == 'AANVAL'] # Annotations with 'SEIZURE' as description
        raw.annotations[raw.annotations.description == 'AANVAL']
        file_path = os.path.join(data_path, file)


        return {
            'Subject ID' : '_'.join(os.path.splitext(file)[0].rsplit('_', 2)[-2:]),
            'Duration' : timedelta(seconds=int(raw.times[-1])),
            'Duration (s)' : int(raw.times[-1]),
            'Number of EEG channels' : sum(raw.ch_names),
            'Channel names' : raw.ch_names,
            'Sampling frequency (Hz)' : len(raw.info['ch_names']),
            'Highpass' : raw.info['highpass'],
            'Lowpass' : raw.info['lowpass'],
            'File size (MB)' : int(os.path.getsize(file_path) / (1024 ** 2))
        }

columns = [
'Subject ID', 'Duration', 'Duration (s)',
'Number of EEG channels', 'Channel names', 'Sampling frequency (Hz)',
'Highpass', 'Lowpass', 'File size (MB)'
]

data_quality_df = pd.DataFrame(columns=columns)

In [ ]:
for file in tqdm(seizure_files):
    file_info = get_eeg_info(file, data_path, seizure_only=True)
    data_quality_df = pd.concat([data_quality_df, pd.DataFrame([file_info])], ignore_index=True)


Pz:

All recordings have 19 EEG channels, except for (insert subject id) as does not include a Pz electrode. This missing data will be filled with the mean value of the other electrodes, as verified with R. van den Berg. Another possibility would have been to interpolate the values based on other electrodes in the Parietal region. This may have been a good option in the case of a “normal” EEG, but since this concerns data with seizures, doing this is not as reliable as just using the mean values of all electrodes. 